In [1]:
import cvxpy as cp, numpy as np

In [2]:
def makeA2(x,y):
    # not used except by makeA1
    u = x.reshape((-1,1))
    v = y.reshape((-1,1))
    return (u*v.T+v*u.T) / 2.  # ip(u,v)

makeA1 = lambda x: makeA2(x,x) # norm(x)

In [88]:
# paramaters

L = 1
sig = 1 # h strong convexity paramater
N = 3
R = 1
phi = (1+np.sqrt(5))/2 # golden ratio
la = (phi/(2*L*sig)) # step-size

In [89]:
dimM = 4*N + 9
dimH = dimG = N + 1
nbPts = N + 2

In [149]:
# encoding vectors

x = np.zeros((nbPts, dimM))  # x^*, x_0, ..., x_N
x_ = np.zeros((nbPts, dimM))
F = np.zeros((nbPts, dimM))
dg = np.zeros((nbPts, dimM))
dh = np.zeros((nbPts, dimM))
dh_ = np.zeros((nbPts, dimM))
h = np.zeros((nbPts, dimH))

dh[:2,-4:-2] = dh_[:2,-2:] = np.eye(2)
x[1:,:nbPts-1] = np.eye(nbPts-1)
x_[1:,nbPts-1:2*nbPts-2] = np.eye(nbPts-1)
F[:,2*nbPts-2:3*nbPts-2] = np.eye(nbPts)
dg[1:,3*nbPts-2:4*nbPts-3] = np.eye(nbPts-1)
dg[0,:] = -F[0,:]

h[1:,]  = np.eye(nbPts-1)
h_ = h.copy()
g = h.copy()
g_ = h.copy()

# encoding GRAAL
p = (phi-1)/(phi) # (phi-1)/phi^2 --> 1.09
q = 1-p
for i in range(N):
    dh[i+2,:] = dh_[i+1,:] - la*(F[i+1,:]+dg[i+1,:])
    dh_[i+2,:] = p*dh[i+2,:] + q*dh_[i+1,:]

Same as MD but with the prox-function g:

$$x_{k+1} = \arg\min_{x\in C} \{\langle F({x}_k), x - x_k\rangle + \frac{1}{\lambda} D_h(x, \bar{x}_k)\} \iff \langle\nabla h(x_{k+1}) - \nabla h(\bar{x}_k) + \lambda F(x_k), x-x_{k+1}\rangle \geq \lambda(g(x_{k+1}) - g(x)) \quad\forall x\in C$$

And the optimallity condition:

$$\langle F(x^*), x - x^*\rangle \geq g(x^*) - g(x) \quad \forall x \in C$$

In [155]:
M = cp.Variable((dimM, dimM), PSD=True)
H = cp.Variable((dimH, 1))
H_ = cp.Variable((dimH, 1))
G = cp.Variable((dimG, 1))
G_ = cp.Variable((dimG, 1))

constraints = {'radius': ((1+phi)*(h[0,:]@H - h_[2,:]@H_ - dh_[2,:] @ M @ (x[0,:] - x_[2,:]))        #(1+phi)D_h(x^*, x_k)
                          + (phi/2)*((h[2,:] - h[1,:])@H - dh[1,:] @ M @ (x[2,:] - x[1,:]))) <= R}   #(phi/2)D_h(x_k, x_{k-1})
               

for i in range(nbPts):
    for j in range(nbPts):
        if i != j:
            constraints[f'Monotonicity @ ({i},{j})'] = (F[i,:] - F[j,:]) @ M @ (x[i,:] - x[j,:]) >= 0
            
            constraints[f'Lipschitz @ ({i}, {j})'] = cp.trace(M@makeA1(F[i,:] - F[j,:])) <= (L**2)*cp.trace(M@makeA1(x[i,:] - x[j,:]))
            
            constraints[f'Strong Convexity @ ({i}, {j})'] = (h[i,:] - h[j,:])@H - dh[j,:] @ M @ (x[i,:] - x[j,:]) >= (sig/2)*cp.trace(M@makeA1(x[i,:] - x[j,:]))
            constraints[f'Strong Convexity @ ({i}, {j}_)'] = h[i,:]@H - h_[j,:]@H_ - dh_[j,:] @ M @ (x[i,:] - x_[j,:]) >= (sig/2)*cp.trace(M@makeA1(x[i,:] - x_[j,:]))
            constraints[f'Strong Convexity @ ({i}_, {j}_)'] = (h_[i,:] - h_[j,:])@H_ - dh_[j,:] @ M @ (x_[i,:] - x_[j,:]) >= (sig/2)*cp.trace(M@makeA1(x_[i,:] - x_[j,:]))
            constraints[f'Strong Convexity @ ({i}_, {j})'] = (h_[i,:] - h[j,:])@H_ - dh[j,:] @ M @ (x_[i,:] - x[j,:]) >= (sig/2)*cp.trace(M@makeA1(x_[i,:] - x[j,:]))
            
        if 0 < i < nbPts-1 and i+1 != j:
            constraints[f'Pythag @ ({i}, {j})'] = (dh_[i,:] - dh[i+1,:] - la*F[i,:]) @ M @ (x[i+1,:] - x[j,:]) >= la*(g[i+1,:] - g[j,:])@G
            
    if i > 0:
        constraints[f'Opt @ ({i})'] = F[0,:] @ M @ (x[i,:] - x[0,:]) >= (g[0,:] - g[i,:])@G
        
    if 0 < i < nbPts-1:
        constraints[f'Decreasing @ ({i})'] = (h[i+1,:]@H - h_[i+1,:]@H_) - dh_[i+1,:]@ M @(x[i+1,:] - x_[i+1,:]) <= phi*(h[i,:]@H - h_[i,:]@H_ - dh_[i,:] @ M @ (x[i,:] - x_[i,:]))
        constraints[f'Assumption @ ({i})'] = (h[i+1,:]@H - h_[i+1,:]@H_) - dh_[i+1,:]@ M @(x[i+1,:] - x_[i+1,:]) <=  (phi-1)*(h[i+1,:]@H - h_[i,:]@H_ - dh_[i,:] @ M @ (x[i+1,:] - x_[i,:]))
#         constraints[f'Assumption @ ({i})'] = ((phi+1)*((h[i+1,:]@H - h_[i+1,:]@H_) - dh_[i+1,:]@ M @(x[i+1,:] - x_[i+1,:]))     # (1+phi)D_h(x_{k+1}, \bar{x}_{k+1})
#                                               - (h[i+1,:]@H - h_[i,:]@H_ - dh_[i,:] @ M @ (x[i+1,:] - x_[i,:]))                 # D_h(x_{k+1}, \bar{x}_k)
#                                               - phi*(h[i,:]@H - h_[i,:]@H_ - dh_[i,:] @ M @ (x[i,:] - x_[i,:]))) <= 0           # phi D_h(x_k, \bar{x}_k)

In [156]:
obj = cp.Maximize((1+phi)*((h[0,:]@H - h_[-1,:]@H_) - dh_[-1,:]@ M @(x[0,:] - x_[-1,:])) # (1+phi)D_h (x^*, \bar{x}_{k+1})
                  + (phi/2)*((h[-1,:] - h[-2,:])@H - dh[-2,:]@ M @(x[-1,:] - x[-2,:])))  # (phi/2)D_h (x_{k+1}, x_k)
prob = cp.Problem(obj, constraints.values())
prob.solve(solver=cp.MOSEK)

0.9999999702122857

In [127]:
# objective
print('PEP value: {:.4f}'.format(obj.value))
# print('Theo value: {:.4f}'.format(R/lam/N))

for name,c in constraints.items():
    if not np.isclose(c.dual_value, 0, atol=1e-3):
        print('\t{}: {:.4f}'.format(name, float(c.dual_value)))

PEP value: 1.3090
	radius: 1.3090
	Monotonicity @ (0,1): 0.0530
	Lipschitz @ (0, 1): 0.0117
	Monotonicity @ (0,2): 0.8597
	Lipschitz @ (0, 2): 0.0822
	Strong Convexity @ (0, 2): 0.4215
	Monotonicity @ (0,3): 0.9062
	Lipschitz @ (0, 3): 0.0722
	Strong Convexity @ (0, 3): 0.2499
	Monotonicity @ (0,4): 0.2841
	Lipschitz @ (0, 4): 0.0131
	Strong Convexity @ (0, 4): 0.1376
	Pythag @ (1, 0): 0.2605
	Monotonicity @ (1,2): 0.0188
	Lipschitz @ (1, 2): 0.4112
	Monotonicity @ (1,3): 0.0728
	Lipschitz @ (1, 3): 0.0170
	Pythag @ (1, 3): 1.3330
	Monotonicity @ (1,4): 0.0425
	Lipschitz @ (1, 4): 0.0147
	Pythag @ (1, 4): 0.2074
	Pythag @ (2, 0): 1.3024
	Strong Convexity @ (2, 1): 1.0590
	Pythag @ (2, 2): 0.6722
	Monotonicity @ (2,3): 0.0245
	Lipschitz @ (2, 3): 0.5190
	Strong Convexity @ (2, 3): 0.5958
	Monotonicity @ (2,4): 0.0453
	Lipschitz @ (2, 4): 0.0783
	Strong Convexity @ (2, 4): 0.2002
	Pythag @ (2, 4): 1.0459
	Opt @ (2): 0.7512
	Assumption @ (2): 1.3090
	Pythag @ (3, 0): 1.1376
	Strong Convex